In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

os.chdir("G:\OMSA\Practicum\\")
current_directory = os.getcwd()
print(current_directory) 

pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)
pd.options.display.max_seq_items = 2000


In [ ]:
# Concatenating all four years of data 
s1 = pd.read_csv(r"G:\OMSA\Practicum\2021\SOURCES2021NEW.csv", encoding='latin-1')
s2 = pd.read_csv(r"G:\OMSA\Practicum\2022\SOURCES2022NEW.csv", encoding='latin-1')
s3 = pd.read_csv(r"G:\OMSA\Practicum\2023\SOURCES2023NEW.csv", encoding='latin-1')
s4 = pd.read_csv(r"G:\OMSA\Practicum\2024\SOURCES2024NEW.csv", encoding='latin-1')

frames = [s1, s2, s3, s4]
sources = pd.concat(frames)
sources.sample(15)
sources.to_csv("SourcesALL.csv", index=False)

In [5]:
len(s1)+len(s2)+len(s3)+len(s4) == len(sources)

True

In [7]:
### Doing some data cleaning
# Adding a Inquiry Opp Type for students with a first source, as these can be assumed to be FTFs
mask = (sources['first_date'].notnull() & sources['inquiry_opp_type'].isnull())
sources['inquiry_opp_type'] =  np.where(mask, "First Time Freshman", sources['inquiry_opp_type'])
#sources.to_csv("TempALL.csv", index=False)

In [9]:
# Standardizing a few rows with incorrect naming for FTF
mask = ((sources['inquiry_opp_type'] == 'First-year undergraduate') | (sources['inquiry_opp_type'] == 'Undergraduate'))
sources['inquiry_opp_type'] =  np.where(mask, "First Time Freshman", sources['inquiry_opp_type'])

In [11]:
sources['inquiry_opp_type'].value_counts()

inquiry_opp_type
First Time Freshman                        208077
Masters                                    117781
Doctoral                                    17301
Transfer                                    16936
Juris Doctor                                  927
Certificate                                   613
Non-Degree                                    563
Readmission                                   132
Graduate (Masters, PhD, EdD, DNP, etc.)        66
Other                                          51
Doctorate                                       7
Master                                          5
Transferring undergraduate                      3
Name: count, dtype: int64

In [13]:
# Dropping all but FTF and TRN rows
sources = sources[sources['inquiry_opp_type'].isin(["First Time Freshman", 'Transfer'])]
sources['inquiry_opp_type'].value_counts()

inquiry_opp_type
First Time Freshman    208077
Transfer                16936
Name: count, dtype: int64

In [15]:
# Counting duplicate rows or IDs
#sources[sources.duplicated('unique_id')].sample(15)
sum(sources.pivot_table(index = ['unique_id'], aggfunc ='size') == 2)

501

In [17]:
# Deduping based on unique_id column and dropping all but the first row from the sources df
sources.sort_values(by='first_date', ascending=True).drop_duplicates(subset=['unique_id'], keep='first', inplace= True)

In [19]:
# Dropping all rows without first source data
sources = sources[sources['first_date'].notnull()]


In [21]:
# Transmuting First Source Stage to binary column that signifies a cold lead (1 = cold, 0 - not cold)
mask = sources['first_source_type'] == 'LEAD SOURCES'
sources['first_source_stage'] =  np.where(mask, 1, 0)
sources.rename(columns={'first_source_stage': 'cold_lead'}, inplace=True)

In [23]:
# Adding Country Info for Vizualization
sources['lead_country'] =  np.where(sources['lead_country'].isnull(), sources['inquiry_mailing_country'], sources['lead_country'])
sources['lead_country'] =  np.where(sources['lead_country'].isnull(), "Unknown", sources['lead_country'])

In [25]:
sum(sources['lead_country'] != "Unknown")

178960

In [37]:
sum(sources['lead_city'].isnull())

127106

In [39]:
# Adding City Info for Vizualization
sources['lead_city'] =  np.where(sources['lead_city'].isnull(), sources['contact_city'], sources['lead_city'])
sources['lead_city'] =  np.where(sources['lead_city'].isnull(), "Unknown", sources['lead_city'])

In [41]:
sources.to_csv("TempALL.csv", index=False)

In [49]:
sources.groupby("Term_Group").lead_country.value_counts(dropna=False)

Term_Group  lead_country                        
Fall 2021   India                                    7606
            China                                    6269
            Unknown                                  6191
            Saudi Arabia                             2575
            Pakistan                                 2221
            United Arab Emirates                     2017
            Canada                                   1919
            Egypt                                    1257
            Lebanon                                  1127
            Mexico                                    936
            Kazakhstan                                687
            South Korea                               670
            Japan                                     516
            Qatar                                     515
            Vietnam                                   417
            Brazil                                    398
            Bangladesh 

In [97]:
sources[sources['Term_Group'] == "Fall 2022"].lead_city.value_counts()

lead_city
Unknown         6750
New Delhi       2285
Cairo           1951
Beijing         1908
Dubai           1635
                ... 
Nandini Town       1
Lille              1
SAS Nagar          1
Uccle              1
Tlaquepaque        1
Name: count, Length: 7395, dtype: int64

In [13]:
sources.columns.values[26:30]

array(['source_type', 'source', 'source_group', 'source_stage'],
      dtype=object)

In [14]:
# Renaming merged columns
sources.columns.values[26:30] = ['source_type_FOS', 'source_FOS', 'source_group_FOS', 'source_stage_FOS']
sources.columns

Index(['Term_Group', 'unique_id', 'lead_id', 'inquiry_id', 'inquiry_opp_id',
       'app_id', 'lead_createddate', 'contact_createddate', 'lead_city',
       'lead_country', 'lead_intlstudent', 'lead_ethnicity', 'lead_gender',
       'contact_city', 'inquiry_home_country', 'inquiry_mailing_country',
       'inquiry_ethnicity', 'inquiry_gender', 'inquiry_langspokenathome',
       'inquiry_opp_type', 'lead_createddate_FLS', 'source_type_FLS',
       'source_FLS', 'source_group_FLS', 'source_stage_FLS',
       'othersource_created_dt', 'source_type_FOS', 'source_FOS',
       'source_group_FOS', 'source_stage_FOS'],
      dtype='object')

In [21]:
sources.rename(columns={'source_type': 'source_type_FES','source':'source_FES', 'source_group': 'source_group_FES'},inplace=True)
#sources.columns.values[35:38] = ['source_type_FES', 'source_FES', 'source_group_FES']
#sources.describe()

In [26]:
df6.shape

(17298, 78)

In [30]:
#df.inquiry_opp_type.value_counts(dropna=False)